# GOAL
Consume talentlms apis and save data to gcs:
- Basic data
    - Call branches api to get branches master data;
    - Persist branches masterdata;
    - Call users api without user id to get users master data;
    - Persist user masterdata;
    - Call courses api without course_id to get courses masterdata;
    - Persist courses master data;
    - Call groups api to get groups master data;
    - Persist group masterdata;
- Detailed data
    - For each active user, call users api for the user_id to get badges, certifications, branches, groups and courses for the user;
    - Persist badges, certifications, branches, groups and courses for the user;
    - For each course in api response, call course api with course_id unit;
    - Persist units by course as course masterdata;
    - For each unit in each course, call unit api to get user data for each unit;
    - Persist user data by unit in course;
    - For each unit that is a test, call test api per enrolled user to get user responses;
    - Persist user response by test;


# PACKAGES

In [1]:
import pandas as pd
import gcsfs
import os
import logging
import seaborn as sns
import talentlms
import yaml
import json
from datetime import date

# PARAMETERS

In [2]:
# gcp
gcp_project = "analytics-dev-308300"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../keys/gcp_key.json"
folder_name='talentlms/'+date.today().strftime('%Y%m%d')

# sns
sns.set(rc={"figure.figsize": (30, 6)})

# talent lms
api_key = yaml.load(open("../keys/talentlms_api_key.yaml"), Loader=yaml.BaseLoader)
domain = "humane.talentlms.com"

# FUNCTIONS

In [3]:
def api_limit(sleep=False):
    limit = lms.ratelimit()
    
    if int(limit['remaining'])!=0:
        credits=int(limit['remaining'])
    elif int(time.time())>int(limit['reset']):
        credits=int(limit['limit'])
    else:
        credits=0
    
    if sleep==True and credits==0:
        print('Waiting for api limit to reset at ',limit['formatted_reset'])
        time.sleep(int(limit['reset'])-int(time.time()))
    
    return credits

In [4]:
def talent2gcs(file_storage,file_path,api_result):
    """
    (obj,str,obj)
    Save api_result as json as a file in gcs under file_path using file_storage object.
    """
    with file_storage.open(file_path,mode='w') as f:
        f.write(json.dumps(api_result))

In [5]:
def call_detail_api(head_data,api_call,api_calls_counter):
    """
    (list of dicts,integer,method,integer)--> list of dicts
    """
    detail=[]
    for item in head_data:
        if api_calls_counter>api_limit():
            api_limit(sleep=True)
        detail.append(api_call(item['id']))
        api_calls_counter+=1
    return detail

In [6]:
def call_detail_api_for_units(course_detail_data,api_calls_counter):
    """
    (list of dicts,integer,method,integer)--> list of dicts
    """
    detail=[]
    set_unit_id = set()
    for item in course_detail_data:
        for unit in item['units']:
            set_unit_id.add(unit['id'])
    for unit_id in set_unit_id:
        if api_calls_counter>api_limit():
            api_limit(sleep=True)
        detail.append(lms.get_user_progress_in_units(unit_id=unit_id))
        api_calls_counter+=1
    return detail

# DATA WRANGLING

## Create talentlms bucket

In [7]:
# open gcs file
fs = gcsfs.GCSFileSystem(project=gcp_project, access="read_write")

In [8]:
try:
    fs.mkdir('talentlms')
except:
    print('bucket already exists')

_request non-retriable exception: You already own this bucket. Please select another name., 409
Traceback (most recent call last):
  File "/home/jmbenedetto/miniconda3/envs/talent-lms-integration/lib/python3.9/site-packages/gcsfs/retry.py", line 110, in retry_request
    return await func(*args, **kwargs)
  File "/home/jmbenedetto/miniconda3/envs/talent-lms-integration/lib/python3.9/site-packages/gcsfs/core.py", line 332, in _request
    validate_response(status, contents, path)
  File "/home/jmbenedetto/miniconda3/envs/talent-lms-integration/lib/python3.9/site-packages/gcsfs/retry.py", line 97, in validate_response
    raise HttpError(error)
gcsfs.retry.HttpError: You already own this bucket. Please select another name., 409


bucket already exists


In [9]:
lms = talentlms.api(domain, api_key["api_key"])

In [10]:
lms.ratelimit()

{'limit': '2000',
 'remaining': '1999',
 'reset': '1625441281',
 'formatted_reset': '04/07/2021, 20:28'}

## Save data from api to gcp

In [11]:
api_calls=4

In [12]:
api_data={
    'branches':lms.branches(),
    'users':lms.users(),
    'courses':lms.courses(),
    'groups':lms.groups(),
}

api_data['users_detail']=call_detail_api(api_data['users'],lms.users,api_calls)
api_data['courses_detail']=call_detail_api(api_data['courses'],lms.courses,api_calls)
api_data['units_detail']=call_detail_api_for_units(api_data['courses_detail'],api_calls)

In [13]:
for api in api_data:
    talent2gcs(fs,folder_name+'/'+api+'.json',api_data[api])

In [22]:
lms.ratelimit()

{'limit': '2000',
 'remaining': 1894,
 'reset': '1625369792',
 'formatted_reset': '04/07/2021, 00:36'}

In [11]:
df_courses=pd.DataFrame(lms.courses())
df_courses

,id,name,code,category_id,description,price,status,creation_date,last_update_on,creator_id,hide_from_catalog,time_limit,expiration_datetime,level,shared,shared_url,avatar,big_avatar,certification,certification_duration
0,124,Advanced Features of TalentLMS,002,None,How do you become a TalentLMS super-user? With...,R&#36;0.00,active,"21/02/2021, 20:16:21","21/02/2021, 20:16:21",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,Fancy,Forever
1,46,Content and TalentLMS,003,None,"When creating an online course, content is all...",R&#36;0.00,active,"21/02/2021, 20:16:21","02/07/2021, 18:48:26",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,None,None
2,113,Employee Training 101,005,None,Investing in employees can be the smartest dec...,R&#36;0.00,active,"21/02/2021, 20:16:21","21/02/2021, 20:16:21",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,None,None
3,122,Getting Started With eLearning,004,None,This course raises some fundamental eLearning ...,R&#36;0.00,active,"21/02/2021, 20:16:21","21/02/2021, 20:16:21",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,None,None
4,123,Introduction to TalentLMS,001,None,Welcome to TalentLMS! Using new software can b...,R&#36;0.00,active,"21/02/2021, 20:16:21","21/02/2021, 20:16:21",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,Fancy,Forever
5,125,test,t1,None,,R&#36;0.00,active,"08/05/2021, 23:57:19","04/07/2021, 14:49:58",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/images/u...,https://d3j0t7vrtr92dk.cloudfront.net/images/u...,None,None
6,126,test 2,,12,,R&#36;0.00,inactive,"04/07/2021, 17:36:27","04/07/2021, 17:36:27",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/images/u...,https://d3j0t7vrtr92dk.cloudfront.net/images/u...,None,None
7,114,This is a SCORM Example Course,006,None,"SCORM is the fastest, easiest way to make your...",R&#36;0.00,active,"21/02/2021, 20:16:21","21/02/2021, 20:16:21",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,None,None


,id,name,code,category_id,description,price,status,creation_date,last_update_on,creator_id,hide_from_catalog,time_limit,expiration_datetime,level,shared,shared_url,avatar,big_avatar,certification,certification_duration
0,124,Advanced Features of TalentLMS,002,None,How do you become a TalentLMS super-user? With...,R&#36;0.00,active,"21/02/2021, 20:16:21","21/02/2021, 20:16:21",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,Fancy,Forever
1,46,Content and TalentLMS,003,None,"When creating an online course, content is all...",R&#36;0.00,active,"21/02/2021, 20:16:21","02/07/2021, 18:48:26",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,None,None
2,113,Employee Training 101,005,None,Investing in employees can be the smartest dec...,R&#36;0.00,active,"21/02/2021, 20:16:21","21/02/2021, 20:16:21",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,None,None
3,122,Getting Started With eLearning,004,None,This course raises some fundamental eLearning ...,R&#36;0.00,active,"21/02/2021, 20:16:21","21/02/2021, 20:16:21",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,None,None
4,123,Introduction to TalentLMS,001,None,Welcome to TalentLMS! Using new software can b...,R&#36;0.00,active,"21/02/2021, 20:16:21","21/02/2021, 20:16:21",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,Fancy,Forever
5,125,test,t1,None,,R&#36;0.00,active,"08/05/2021, 23:57:19","04/07/2021, 14:49:58",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/images/u...,https://d3j0t7vrtr92dk.cloudfront.net/images/u...,None,None
6,114,This is a SCORM Example Course,006,None,"SCORM is the fastest, easiest way to make your...",R&#36;0.00,active,"21/02/2021, 20:16:21","21/02/2021, 20:16:21",1,0,0,None,None,0,,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,https://d3j0t7vrtr92dk.cloudfront.net/sampleco...,None,None


In [9]:
df_courses.iloc[5]

id                                                                      125
name                                                                   test
code                                                                     t1
category_id                                                            None
description                                                                
price                                                            R&#36;0.00
status                                                               active
creation_date                                          08/05/2021, 23:57:19
last_update_on                                         04/07/2021, 14:49:58
creator_id                                                                1
hide_from_catalog                                                         0
time_limit                                                                0
expiration_datetime                                                    None
level       

In [12]:
course=lms.courses(124)
course

{'id': '124',
 'name': 'Advanced Features of TalentLMS',
 'code': '002',
 'category_id': None,
 'description': 'How do you become a TalentLMS super-user? With this brief training on some of the more advanced features of our platform. Dive in and see how you can save time when building a course.',
 'price': 'R&#36;0.00',
 'status': 'active',
 'creation_date': '21/02/2021, 20:16:21',
 'last_update_on': '21/02/2021, 20:16:21',
 'creator_id': '1',
 'hide_from_catalog': '0',
 'time_limit': '0',
 'expiration_datetime': None,
 'level': None,
 'shared': '0',
 'shared_url': '',
 'avatar': 'https://d3j0t7vrtr92dk.cloudfront.net/samplecourses/1548346699_toolkit.png?',
 'big_avatar': 'https://d3j0t7vrtr92dk.cloudfront.net/samplecourses/1548346701_toolkit.png?',
 'certification': 'Fancy',
 'certification_duration': 'Forever',
 'users': [{'id': '1',
   'name': 'J. Benedetto',
   'role': 'instructor',
   'enrolled_on': '21/02/2021, 20:16:21',
   'enrolled_on_timestamp': '1613949381',
   'completed_on

# SCRIPT

In [ ]:
# open gcs file
fs = gcsfs.GCSFileSystem(project=gcp_project, access="read_write")
# create folder in gcs if it does not already exist
try:
    fs.mkdir('talentlms')
except:
    print('bucket already exists')
    
# instantiate talentlms api connector
lms = talentlms.api(domain, api_key["api_key"])

# call apis
api_data={
    'branches':lms.branches(),
    'users':lms.users(),
    'courses':lms.courses(),
    'groups':lms.groups(),
}
api_data['users_detail']=call_detail_api(api_data['users'],lms.users,api_calls)
api_data['courses_detail']=call_detail_api(api_data['courses'],lms.courses,api_calls)
api_data['units_detail']=call_detail_api_for_units(api_data['courses_detail'],api_calls)

# save data to gcs
for api in api_data:
    talent2gcs(fs,folder_name+'/'+api+'.json',api_data[api])